Create Azure SQL Virtual Machine
============================================

Description
-----------
For more information about other Azure PowerShell options for creating SQL VMs, see the <a href="https://docs.microsoft.com/en-us/azure/virtual-machines/windows/sql/virtual-machines-windows-ps-sql-create">Provisioning guide for SQL Server VMs with Azure PowerShell</a>. See also <a href="https://docs.microsoft.com/en-us/azure/virtual-machines/windows/sql/quickstart-sql-vm-create-powershell">Quickstart guide</a> to creating a SQL Server VM with Azure PowerShell.


Steps of this procedure include:
1. Connect to Azure subscription
1. Provision resource group for SQL VM migration
2. Create a storage account
3. Configure Network Settings
3. Provision SQL VM
4. Configure SQL VM IaaS agent

## Notebook setup

In [ ]:
import pandas, sys, os, json, html, getpass, time, tempfile, string, random, subprocess

def run_command(command, json_decode = True):
    print(command)
    process = subprocess.Popen(command.split(), stdout=subprocess.PIPE, stderr=subprocess.PIPE, shell=True)
    output, error = process.communicate()
    if process.returncode != 0: 
        print("Process failed %d \n%s" % (process.returncode, error.decode("utf-8")))
        raise Exception()
    if output:
        output = output.decode("utf-8")
        print(output)
        try:
            return json.loads(output)
        except:
            return output

def get_random_string(length):
    letters = string.ascii_lowercase
    result_str = ''.join(random.choice(letters) for i in range(length))
    print("Random string of length", length, "is:", result_str)
    return result_str

## Set variables

In [ ]:
if "AZDATA_NB_VAR_AZURE_SQLVM_PASSWORD" in os.environ:
    azure_sqlvm_password = os.environ["AZDATA_NB_VAR_AZURE_SQLVM_PASSWORD"]

## Connect to Azure Account

In [ ]:
run_command('azdata --version', False);

## Set Subscription
Below command will open a _Dialouge Box_ with list of subscriptions. Selecting one of those will set that subscription for rest of the commands.

In [ ]:
run_command(
    'az account set '
    '--subscription {0}'
    .format(
        azure_sqlvm_nb_var_subscription));

## Create a storage account
VMs require storage resources for OS, SQL data and logs. Create a new storage account as a place for it.

In [ ]:
storage_account_check = run_command('az storage account check-name --name {0}'
.format(azure_sqlvm_storageaccountname))



if storage_account_check['nameAvailable']:
    create_storage_account_result = run_command(
        'az storage account create '
        '--name {0} '
        '--resource-group {1} '
        '--sku {2} '
        '--kind Storage '
        '--location {3}'
        .format(
            azure_sqlvm_storageaccountname, 
            azure_sqlvm_nb_var_resource_group_name, 
            azure_sqlvm_storagesku, 
            azure_sqlvm_location));

## Configure network settings
Create a virtual network, subnet, and a public IP address. These resources are used to provide network connectivity to the virtual machine and connect it to the internet.

In [ ]:
subnet_name = azure_sqlvm_nb_var_resource_group_name + "subnet"
vnet_name = azure_sqlvm_nb_var_resource_group_name + "vnet"
pip_name = azure_sqlvm_nb_var_resource_group_name + get_random_string(5)

run_command(
   'az network vnet create '
   '--resource-group {0} '
   '--name {1} '
   '--location {2} '
   '--address-prefixes 192.168.0.0/16 '
   .format(
      azure_sqlvm_nb_var_resource_group_name, 
      vnet_name, 
      azure_sqlvm_location));

run_command(
   'az network vnet subnet create '
   '--name {0} '
   '--resource-group {1} '
   '--vnet-name {2} '
   '--address-prefixes 192.168.1.0/24'
   .format(
      subnet_name,
       azure_sqlvm_nb_var_resource_group_name,
       vnet_name
       ));

run_command(
   'az network public-ip create '
   '--resource-group {0} '
   '--location {1} '
   '--allocation-method Static '
   '--idle-timeout 4 '
   '--name {2}'
   .format(
      azure_sqlvm_nb_var_resource_group_name, 
      azure_sqlvm_location, 
      pip_name));

2. Create a network security group. Configure rules to allow remote desktop (RDP) and SQL Server connections.

In [ ]:
nsg_name = azure_sqlvm_nb_var_resource_group_name + 'nsg'

run_command(
   'az network nsg create '
   '--resource-group {0} '
   '--location {1} '
   '--name {2}'
   .format(
      azure_sqlvm_nb_var_resource_group_name,
      azure_sqlvm_location,
      nsg_name));


run_command(
   'az network nsg rule create '
   '--name RDPRule '
   '--nsg-name {0} '
   '--priority 1000 '
   '--resource-group {1} '
   '--protocol Tcp '
   '--direction Inbound '
   '--source-address-prefixes * '
   '--source-port-range * '
   '--destination-address-prefixes * '
   '--destination-port-range 3389 '
   '--access Allow'
   .format(
      nsg_name,
      azure_sqlvm_nb_var_resource_group_name));

run_command(
   'az network nsg rule create '
   '--name MSSQLRule '
   '--nsg-name {0} '
   '--priority 1001 '
   '--resource-group {1} '
   '--protocol Tcp '
   '--direction Inbound '
   '--source-address-prefixes * '
   '--source-port-range * '
   '--destination-address-prefixes * '
   '--destination-port-range 1433 '
   '--access Allow'
   .format(
      nsg_name,
      azure_sqlvm_nb_var_resource_group_name));

3. Create the network interface.

In [ ]:
interface_name = azure_sqlvm_nb_var_resource_group_name + "int"

run_command(
   'az network nic create '
   '--name {0} '
   '--resource-group {1} '
   '--location {2} '
   '--subnet {3} '
   '--vnet-name {4} '
   '--public-ip-address {5} '
   '--network-security-group {6}'
   .format(
      interface_name,
      azure_sqlvm_nb_var_resource_group_name,
      azure_sqlvm_location,
      subnet_name,
      vnet_name,
      pip_name,
      nsg_name));

## Create Virtual Machine

In [ ]:
# Create the VM
run_command(
   'az vm create '
   '--name {0} '
   '--size {1} '
   '--computer-name {0} '
   '--admin-username {2} '
   '--admin-password {3} '
   '--image {4}:{5}:{6}:{7} '
   '--nics {8} '
   '--resource-group {9} '
   '--location {10} '
   .format(
       azure_sqlvm_vmname,
       azure_sqlvm_vmsize,
       azure_sqlvm_username,
       azure_sqlvm_password,
       azure_sqlvm_publisher,
       azure_sqlvm_image,
       azure_sqlvm_image_sku,
       'latest',
       interface_name,
       azure_sqlvm_nb_var_resource_group_name,
       azure_sqlvm_location
   )
);

## Install the SQL IaaS Agent
To get portal integration and SQL VM features, you must install the SQL Server IaaS Agent Extension. To install the agent on the new VM, run the following command after the VM is created.

In [ ]:
run_command(
    'az vm extension set '
    '--vm-name {0} '
    '--resource-group {1} '
    '--name SqlIaaSAgent '
    '--publisher Microsoft.SqlServer.Management '
    .format(
        azure_sqlvm_vmname,
        azure_sqlvm_nb_var_resource_group_name    
    )
);